# Skills Vectorisation

In [8]:
import re
from google.colab import drive
import pandas as pd
import os

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet') 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
drive.mount('/content/gdrive')
drive_path = '/content/gdrive/MyDrive/IS4242'
os.chdir(drive_path)
print(os.getcwd())

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/.shortcut-targets-by-id/1CDppNxm5AHmRDwItmGGQo-T6INF5HrH_/IS4242


In [11]:
# drive_items = os.listdir(os.getcwd())
# print(drive_items)

In [29]:
#df = pd.read_csv('Supervised_Component/quickFE.csv')
data_scientist_us_df = pd.read_csv('Supervised_Component/data_scientist_us.csv', encoding='iso-8859-1')
data_scientist_sg_df = pd.read_csv('Supervised_Component/data_scientist_sg.csv', encoding='iso-8859-1')
data_analyst_us_df = pd.read_csv('Supervised_Component/data_analyst_us.csv', encoding='iso-8859-1')
data_analyst_sg_df = pd.read_csv('Supervised_Component/data_analyst_sg.csv', encoding='iso-8859-1')
data_engineer_us_df = pd.read_csv('Supervised_Component/data_engineer_us.csv', encoding='iso-8859-1')
data_engineer_sg_df = pd.read_csv('Supervised_Component/data_engineer_sg.csv', encoding='iso-8859-1')
df_fe = pd.read_csv('Supervised_Component/final_dataset.csv', encoding='iso-8859-1') 

In [30]:
df_fe.columns

Index(['company_starRating', 'company_salary', 'listing_jobDesc', 'role_type',
       'location', 'Monthly_Salary', 'cleaned_listing_jobDesc',
       'country_binarylabel', 'tableau', 'data warehousing',
       'machine learning', 'etl', 'power bi', 'scala', 'excel', 'hadoop',
       'oracle', 'r', 'data pipeline', 'spark', 'sql', 'data governance',
       'data engineering', 'snowflake', 'optimization', 'git', 'powerpoint',
       'nosql', 'data quality', 'data visualization', 'data mining', 'aws',
       'business intelligence', 'data engineer', 'agile', 'java', 'rest',
       'query', 'data analyst', 'python', 'data scientist',
       'project management', 'big data', 'data modeling', 'azure', 'api',
       'unity'],
      dtype='object')

In [31]:
print("--data analyst--")
data_analyst_us_df_2 = data_analyst_us_df.drop_duplicates(subset=['listing_jobDesc'])#[:150]
data_analyst_sg_df_2 = data_analyst_sg_df.drop_duplicates(subset=['listing_jobDesc'])#[:150]
print("US total: {}, Unique JD: {}".format(len(data_analyst_us_df), len(data_analyst_us_df_2)))
print("SG total: {}, Unique JD: {}".format(len(data_analyst_sg_df), len(data_analyst_sg_df_2)))

print("--data scientist--")
data_scientist_us_df_2 = data_scientist_us_df.drop_duplicates(subset=['listing_jobDesc'])#[:150]
data_scientist_sg_df_2 = data_scientist_sg_df.drop_duplicates(subset=['listing_jobDesc'])#[:150]
print("US total: {}, Unique JD: {}".format(len(data_scientist_us_df), len(data_scientist_us_df_2)))
print("SG total: {}, Unique JD: {}".format(len(data_scientist_sg_df), len(data_scientist_sg_df_2)))

print("--data engineer--")
data_engineer_us_df_2 = data_engineer_us_df.drop_duplicates(subset=['listing_jobDesc'])#[:150]
data_engineer_sg_df_2 = data_engineer_sg_df.drop_duplicates(subset=['listing_jobDesc'])#[:150]
print("US total: {}, Unique JD: {}".format(len(data_engineer_us_df), len(data_engineer_us_df_2)))
print("SG total: {}, Unique JD: {}".format(len(data_engineer_sg_df), len(data_engineer_sg_df_2)))

--data analyst--
US total: 4678, Unique JD: 1790
SG total: 1003, Unique JD: 381
--data scientist--
US total: 609, Unique JD: 609
SG total: 1038, Unique JD: 246
--data engineer--
US total: 2398, Unique JD: 995
SG total: 1875, Unique JD: 876


In [32]:
data_scientist_us_df_2 = data_scientist_us_df_2[:150]
data_scientist_sg_df_2 = data_scientist_sg_df_2[:150]
data_analyst_us_df_2 = data_analyst_us_df_2[:150]
data_analyst_sg_df_2 = data_analyst_sg_df_2[:150]
data_engineer_us_df_2 = data_engineer_us_df_2[:150]
data_engineer_sg_df_2 = data_engineer_sg_df_2[:150]

combined_df = pd.concat([data_scientist_us_df_2,data_scientist_sg_df_2,data_analyst_us_df_2, data_analyst_sg_df_2, data_engineer_us_df_2, data_engineer_sg_df_2])

In [33]:
len(combined_df)

900

In [34]:
print("--combined--")
combined_df_2 = combined_df.drop_duplicates(subset=['listing_jobDesc'])
print("Combined total: {}, Unique JD: {}".format(len(combined_df), len(combined_df_2)))

--combined--
Combined total: 900, Unique JD: 845


# Zero Shot Classification

In [35]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [36]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=0)

# pipe = pipeline(model="facebook/bart-large-mnli")
# pipe("I have a problem with my iphone that needs to be resolved asap!",
#     candidate_labels=["urgent", "not urgent", "phone", "tablet", "computer"],
# )

## Example

In [37]:
job_description = combined_df_2.iloc[0]['listing_jobDesc']
# skills_lst = ['sql','python','nosql','postgresql','redshift','athena','tableau','power bi','excel','data visualization','dashboards','machine learning','scala','java','javascript','html','c++','hive','aws','tensorflow','pytorch','flask','plumber','ai','keras','matlab','data scraping', 'nlp','powerpoint','etl','azure','linux','spark','hadoop','docker','kubernetes','numpy','postgresql','query']
# results = classifier(job_description, skills_lst)
# print(results)

### Very slow
1. Reduce size of skill list
2. Use smaller model, eg BERT based
3. Shorten the JD. remove stop words,stemming/ lemmatization

In [38]:
def job_description_preprocessing(job_description):
  job_description = job_description.lower()
  job_description = ''.join(c for c in job_description if c.isalnum() or c.isspace())
  words = word_tokenize(job_description)
  stop_words = set(stopwords.words('english'))
  words = [word for word in words if word not in stop_words]
  lemmatizer = WordNetLemmatizer()
  words = [lemmatizer.lemmatize(word) for word in words]
  return ' '.join(words)

In [39]:
def get_skills_vectors(job_description, skills_lst):
  result = classifier(job_description, skills_lst, multi_label=True)
  skills_dict = dict(zip(result['labels'], result['scores']))
  return skills_dict

In [40]:
job_description = job_description_preprocessing(job_description)
print(job_description)

execute sampling weighting plan various probability nonprobability survey effort ranging regional survey smaller survey effort targeting specific population perform analysis large data set translate finding digestible information help inform various agency project research collect compile data ensure accuracy information maintain database documentation perform complex statistical method analysis quasiexperimental research design nonequivalent group eg regression adjustment matching propensity score stratification longitudinal analysis classification dimension reduction clustering hierarchical linear random effect modeling various evaluation project perform various duty related development maintenance improvement sandags abm rsm cvm cbm perform research participate development modeling feature related emerging transportation technology transformative mode shared mobility ensure transportation modeling tool analytical capability credible effective policy analysis participate quality assu

In [41]:
skills_lst = [
    'tableau', 'data warehousing',
    'machine learning', 'etl', 'power bi', 
    'scala', 'excel', 'hadoop', 'oracle', 
    'r', 'data pipeline', 'spark', 'sql', 
    'data governance', 'data engineering', 
    'snowflake', 'optimization', 'git', 
    'powerpoint', 'nosql', 'data quality', 
    'data visualization', 'data mining', 
    'aws', 'business intelligence', 
    'data engineer', 'agile', 'java', 
    'rest', 'query', 'data analyst', 
    'python', 'data scientist', 
    'project management', 'big data', 
    'data modeling', 'azure', 'api', 
    'unity'
    ]
       
print(len(skills_lst))
results_2 = classifier(job_description, skills_lst, multi_label=True)

39


In [42]:
results_2['sequence']

'execute sampling weighting plan various probability nonprobability survey effort ranging regional survey smaller survey effort targeting specific population perform analysis large data set translate finding digestible information help inform various agency project research collect compile data ensure accuracy information maintain database documentation perform complex statistical method analysis quasiexperimental research design nonequivalent group eg regression adjustment matching propensity score stratification longitudinal analysis classification dimension reduction clustering hierarchical linear random effect modeling various evaluation project perform various duty related development maintenance improvement sandags abm rsm cvm cbm perform research participate development modeling feature related emerging transportation technology transformative mode shared mobility ensure transportation modeling tool analytical capability credible effective policy analysis participate quality ass

In [43]:
skills_dict = dict(zip(results_2['labels'], results_2['scores']))
skills_dict

{'azure': 0.9887955188751221,
 'data visualization': 0.9819573163986206,
 'tableau': 0.9804214239120483,
 'data modeling': 0.9799231886863708,
 'power bi': 0.9796057939529419,
 'java': 0.9790242910385132,
 'project management': 0.9774934649467468,
 'query': 0.972189724445343,
 'python': 0.9712331295013428,
 'r': 0.9672112464904785,
 'sql': 0.9653732776641846,
 'spark': 0.9614583849906921,
 'data governance': 0.9581291079521179,
 'data quality': 0.9547718167304993,
 'etl': 0.9406257271766663,
 'data analyst': 0.9336547255516052,
 'big data': 0.9272109866142273,
 'aws': 0.9171643853187561,
 'data scientist': 0.886657178401947,
 'hadoop': 0.8814746737480164,
 'excel': 0.8551022410392761,
 'data engineering': 0.8486582636833191,
 'api': 0.8277952671051025,
 'nosql': 0.8049618601799011,
 'business intelligence': 0.7767814993858337,
 'oracle': 0.7665531039237976,
 'rest': 0.7481712102890015,
 'data engineer': 0.7451900243759155,
 'git': 0.7262341976165771,
 'agile': 0.7113373875617981,
 'uni

# Processing start - Actual

In [44]:
combined_df_2['cleaned_jd'] = combined_df_2['listing_jobDesc'].apply(lambda x: job_description_preprocessing(str(x)))

In [45]:
combined_df_2['skills_vec'] = combined_df_2['cleaned_jd'].apply(lambda x: get_skills_vectors(str(x), skills_lst))

In [46]:
combined_df_2.to_csv('Skills Vector/Skills_Vec_BART_Combined_2.csv', encoding='utf-8', index=False)

# BERT extract keywords

In [ ]:
# job_description = sample_df.iloc[0]['listing_jobDesc']
# job_description = job_description_preprocessing(job_description)

# from sklearn.feature_extraction.text import CountVectorizer

# # length of keyword
# n_gram_range = (1, 1)

# # for removing stopwords
# stop_words = "english"

# # Extract candidate words/phrases
# count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([job_description])
# candidates = count.get_feature_names_out()
# print('Printing candidate keywords from this sample job description: ', candidates)

In [ ]:
# !pip install sentence-transformers

In [ ]:
# from sentence_transformers import SentenceTransformer

# model = SentenceTransformer('distilbert-base-nli-mean-tokens')
# doc_embedding = model.encode([job_description])
# #candidates = ['sql', 'python', 'nosql', 'postgresql', 'tableau', 'excel', 'data visualization', 'machine learning', 'java', 'javascript', 'html', 'hive', 'aws', 'tensorflow', 'pytorch', 'nlp', 'etl', 'azure', 'linux', 'spark', 'hadoop', 'docker', 'numpy']
# candidates = ['sql', 'python', 'java', 'extract transform load (etl)', 'data modeling', 'data warehousing', 'data visualization',
#                'excel', 'tableau', 'statistical analysis', 'regression analysis', 'machine learning',
#                'deep learning', 'natural language processing', 'big data technologies', 'hadoop',
#                'spark', 'amazon web services (aws)', 'azure', 'google cloud platform (gcp)', 'git', 'mathematics', 'statistics', 'cloud computing',
#                'computer vision', 'tensorflow', 'pytorch'
#                ]
# candidate_embeddings = model.encode(candidates)

In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity

# distances = cosine_similarity(doc_embedding, candidate_embeddings)
# keywords = [candidates[index] for index in distances.argsort()[0]]